<a href="https://colab.research.google.com/github/wesleyc00/spam-detector/blob/main/Spam_Detector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Carregar dataset

In [1]:
import pandas as pd
# Carregar arquivo txt
spam_df = pd.read_csv('dataset_spam_pt_br.txt', sep=',')
spam_df.head()

,Categoria,Mensagem
0,ham,"Oi, como você está?"
1,spam,Você ganhou um prêmio! Clique aqui para resgatar.
2,ham,Estou disponível para nossa reunião amanhã.
3,spam,"Oferta exclusiva! 90% de desconto, aproveite a..."
4,ham,Poderíamos nos encontrar para discutir o projeto?


# Converter Rótulos para Numéricos

In [2]:
# Convertendo rótulos para valores numéricos
spam_df['Categoria'] = spam_df['Categoria'].map({'ham': 0, 'spam': 1}) # 0 = ham (Não spam); 1 = spam.

print(spam_df.head())

   Categoria                                           Mensagem
0          0                                Oi, como você está?
1          1  Você ganhou um prêmio! Clique aqui para resgatar.
2          0        Estou disponível para nossa reunião amanhã.
3          1  Oferta exclusiva! 90% de desconto, aproveite a...
4          0  Poderíamos nos encontrar para discutir o projeto?


# Pré-Processamento dos Dados

In [3]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

nltk.download('stopwords')
stop_words = set(stopwords.words('portuguese'))
stemmer = PorterStemmer()

def preprocess_text(text):
    text = text.lower()  # Converter para minúsculas
    text = re.sub(r'\W', ' ', text)  # Remover caracteres especiais
    text = re.sub(r'\s+', ' ', text).strip()  # Remover espaços extras
    words = text.split()
    words = [stemmer.stem(word) for word in words if word not in stop_words]
    return ' '.join(words)

spam_df['processed_message'] = spam_df['Mensagem'].apply(preprocess_text)

print(spam_df)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


     Categoria                                           Mensagem  \
0            0                                Oi, como você está?   
1            1  Você ganhou um prêmio! Clique aqui para resgatar.   
2            0        Estou disponível para nossa reunião amanhã.   
3            1  Oferta exclusiva! 90% de desconto, aproveite a...   
4            0  Poderíamos nos encontrar para discutir o projeto?   
..         ...                                                ...   
286          1                      Temos novidades no Inter Loop   
287          0                           Envio da NFe - Emitente:   
288          1  Quase. Quase. Está quase acabando | Ofertas do...   
289          0               Seu relatório de ganhos ficou pronto   
290          1         A performance e os preços que você queria.   

                               processed_message  
0                                             oi  
1              ganhou prêmio cliqu aqui resgatar  
2                 

# Converter Texto em Vetores

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(max_features=5000)
X = vectorizer.fit_transform(spam_df['processed_message']).toarray()
y = spam_df['Categoria']

# Treinar ML

In [5]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report

# Dividir os dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Treinar o modelo
model = MultinomialNB()
model.fit(X_train, y_train)

# Fazer previsões
y_pred = model.predict(X_test)

# Avaliação do modelo
print("Acurácia:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

Acurácia: 0.9204545454545454
              precision    recall  f1-score   support

           0       0.91      0.93      0.92        44
           1       0.93      0.91      0.92        44

    accuracy                           0.92        88
   macro avg       0.92      0.92      0.92        88
weighted avg       0.92      0.92      0.92        88



# Testar com novas mensagens

In [6]:
def predict_spam(message):
    processed_message = preprocess_text(message)
    vectorized_message = vectorizer.transform([processed_message]).toarray()
    prediction = model.predict(vectorized_message)
    return "Spam" if prediction[0] == 1 else "Não spam"

# Teste
print(predict_spam("Olá, tudo bem"))

Não spam


In [7]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report

# Baixar stopwords e lematizador
nltk.download('stopwords')
nltk.download('wordnet')

# Carregar dataset
spam_df = pd.read_csv('dataset_spam_pt_br.txt', sep=',')
spam_df['Categoria'] = spam_df['Categoria'].map({'ham': 0, 'spam': 1})

# Inicializar ferramentas de NLP
stop_words = set(stopwords.words('portuguese'))  # Stopwords em português
lemmatizer = WordNetLemmatizer()

# Função de pré-processamento de texto
def preprocess_text(text):
    text = text.lower()  # Converter para minúsculas
    text = re.sub(r'http\S+|www\S+', '', text)  # Remover URLs
    text = re.sub(r'\d+', '', text)  # Remover números
    text = re.sub(r'\W', ' ', text)  # Remover caracteres especiais
    text = re.sub(r'\s+', ' ', text).strip()  # Remover espaços extras
    words = text.split()
    words = [lemmatizer.lemmatize(word) for word in words if word not in stop_words]
    return ' '.join(words)

# Aplicar pré-processamento
spam_df['processed_message'] = spam_df['Mensagem'].apply(preprocess_text)

# Vetorização TF-IDF com N-grams
vectorizer = TfidfVectorizer(max_features=5000, ngram_range=(1,2))
X = vectorizer.fit_transform(spam_df['processed_message']).toarray()
y = spam_df['Categoria']

# Dividir dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Treinar modelo SVM
model = SVC(kernel='linear', C=1.0)
model.fit(X_train, y_train)

# Fazer previsões
y_pred = model.predict(X_test)

# Avaliação do modelo
print("Acurácia:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

# Função para prever mensagens novas
def predict_spam(message):
    processed_message = preprocess_text(message)
    vectorized_message = vectorizer.transform([processed_message]).toarray()
    prediction = model.predict(vectorized_message)
    return "Spam" if prediction[0] == 1 else "Não Spam"

# Teste
print(predict_spam("Olá amigo, aproveite as ofertas da semana"))


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...


Acurácia: 0.8977272727272727
              precision    recall  f1-score   support

           0       0.86      0.95      0.90        44
           1       0.95      0.84      0.89        44

    accuracy                           0.90        88
   macro avg       0.90      0.90      0.90        88
weighted avg       0.90      0.90      0.90        88

Spam


# Teste com Random Forest

In [8]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

# Baixar stopwords e lematizador
nltk.download('stopwords')
nltk.download('wordnet')

# Carregar dataset
spam_df = pd.read_csv('dataset_spam_pt_br.txt', sep=',')
spam_df['Categoria'] = spam_df['Categoria'].map({'ham': 0, 'spam': 1})

# Inicializar ferramentas de NLP
stop_words = set(stopwords.words('portuguese'))  # Stopwords em português
lemmatizer = WordNetLemmatizer()

# Função de pré-processamento de texto
def preprocess_text(text):
    text = text.lower()  # Converter para minúsculas
    text = re.sub(r'http\S+|www\S+', '', text)  # Remover URLs
    text = re.sub(r'\d+', '', text)  # Remover números
    text = re.sub(r'\W', ' ', text)  # Remover caracteres especiais
    text = re.sub(r'\s+', ' ', text).strip()  # Remover espaços extras
    words = text.split()
    words = [lemmatizer.lemmatize(word) for word in words if word not in stop_words]
    return ' '.join(words)

# Aplicar pré-processamento
spam_df['processed_message'] = spam_df['Mensagem'].apply(preprocess_text)

# Vetorização TF-IDF com N-grams
vectorizer = TfidfVectorizer(max_features=5000, ngram_range=(1,2))
X = vectorizer.fit_transform(spam_df['processed_message']).toarray()
y = spam_df['Categoria']

# Dividir dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Treinar modelo Random Forest
model = RandomForestClassifier(n_estimators=200, random_state=42)
model.fit(X_train, y_train)

# Fazer previsões
y_pred = model.predict(X_test)

# Avaliação do modelo
print("Acurácia:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

# Função para prever mensagens novas
def predict_spam(message):
    processed_message = preprocess_text(message)
    vectorized_message = vectorizer.transform([processed_message]).toarray()
    prediction = model.predict(vectorized_message)
    return "Spam" if prediction[0] == 1 else "Não Spam"

# Teste
print(predict_spam("Aproveite nossas promoções de verão!"))


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Acurácia: 0.7613636363636364
              precision    recall  f1-score   support

           0       0.68      1.00      0.81        44
           1       1.00      0.52      0.69        44

    accuracy                           0.76        88
   macro avg       0.84      0.76      0.75        88
weighted avg       0.84      0.76      0.75        88

Não Spam
